# Project 2b
####  Use NLTK •Sample of text we are processing: •
`This movie made it into one of my top 10 most awful movies. Horrible. There wasn't a continuous minute where there wasn't a fight with one monster or another. There was no chance for any character development, they were too busy running from one sword fight to another. I had no emotional attachment ( except to the big bad machine ## that wanted to destroy them). •Write python code to clean the above text using the textual data cleaning methods discussed earlier and implement all the strategies as functions. •Using the functions created for cleaning generate summary of the above data by applying TF/IDF scoring.`

In [38]:
#importing Libraries
!pip install contractions
import contractions 
import nltk
from nltk.corpus import stopwords
import re
from nltk.tokenize import sent_tokenize, word_tokenize  # ['i', 'play', 'cricket']
import numpy as np
import pandas as pd
import string
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk import tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

nltk.download('maxent_ne_chunker')
nltk.download("words")
# installing stopwords.
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is alrea

True

In [39]:
import re
from nltk.tokenize import sent_tokenize, word_tokenize

In [40]:
sentence = "This movie made it into one of my top 10 most awful movies. Horrible. I don’t care if it makes 1 million, 10 M , or 100. There wasn't a continuous minute where there wasn't a fight with one monster or another. There was no chance for any character development, they were too busy running from one sword fight to another. I had no emotional attachment ( except to the big bad machine ## that wanted to destroy them). If you disagree with me, you can send your thoughts to idonotcare@leavemealone.com"

In [41]:
print(sentence)

This movie made it into one of my top 10 most awful movies. Horrible. I don’t care if it makes 1 million, 10 M , or 100. There wasn't a continuous minute where there wasn't a fight with one monster or another. There was no chance for any character development, they were too busy running from one sword fight to another. I had no emotional attachment ( except to the big bad machine ## that wanted to destroy them). If you disagree with me, you can send your thoughts to idonotcare@leavemealone.com


In [42]:
#creating a class for text pre processing
class Text_Wrap(object):  #nik = Text_Wrap(st)
    def __init__ (self,st):
        self.st= st
        self.tokens = []
        self.filtered= []

    def reg_exp(self):
        """ This function will perform pre-processing on the sample text by lowering the text, remove the numbers and special characters 
            
            :return self.st = precprocessed text
        """

        #self.st = self.st.lower()
        self.st = re.sub('\d+',' number',self.st)   #1 million -> number million                         #substituting digits with 'numbers'
        self.st = re.sub('\w[!-?A-~]*@\w[\w\.]*\w','email_id', self.st)     #for emails 
        self.st = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", self.st) #to substitute special characters with ""
        self.st = re.sub('\s{2,}', ' ', self.st) #remove extra spaces
        return self.st


    def expand_contractions(self):
      """ This function will expand all the contactions i.e expanding the words with apostrophe
      
          :return st.text = st after expanding contactions

      """
      expand_st = []
      for i in self.st.split():
          expand_st.append(contractions.fix(i)) 
      self.st = ' '.join(expand_st)

      return self.st


    def tokenization(self,type):
      """ This function will tokenize the text either sentence or word 

          :return self.tokens = tokens

      """

      if type == 'sentence_tokenize': 
            self.tokens = nltk.tokenize.sent_tokenize(self.st)
      else:
            self.tokens = nltk.tokenize.word_tokenize(self.st)           
        
      return self.tokens  


    def stopword_removal (self, language = 'english'):

      """ This function will remove the english language stopwords

          :return self.filter : st after filtering.
      """
      stop_words = set(stopwords.words(language))     #   ['switch', 'off','the',  'light']     ['the','a','an','this']                    
      self.filter = [i for i in self.tokens if not i.lower() in stop_words] 
      
      return self.filter #['switch', 'off', 'light']

    def text_normalize(self,t_type):
      """
      This function will either perform Stemming or Lemmatization

      :return self.text

      """
      if t_type == 'stem':   
          ps = PorterStemmer()
          self.text = [ps.stem(i) for i in self.filter]             
      elif t_type == 'lemm':
          lem = WordNetLemmatizer()
          self.text = [lem.lemmatize(i) for i in self.filter]        
      
      return self.text    


    def Result(self):
      """
      This function will get the final result 
      """
      return self.text

    def tfidf(self, corpus_texts):
        """
        This method will apply the TFIDF to the list of sentences
        in the arguments and return the vectors.

        :return sentences_vectors: Numpy array of vectors
        """
        wordfreq = {}
        for sentence in corpus_texts:
            # sent_processing = ProcessingText(sentence)
            self.st = sentence
            processed_sent = self.complete_processing()
            for token in processed_sent:
                if token not in wordfreq.keys():
                    wordfreq[token] = 1
                else:
                    wordfreq[token] += 1
        vocab = wordfreq
        # tf and idf
        termDict={}
        docsTFMat = np.zeros((len(corpus_texts),len(vocab)))
        docsIdfMat = np.zeros((len(vocab),len(corpus_texts)))

        docTermDf = pd.DataFrame(docsTFMat ,columns=sorted(vocab.keys()))
        docCount=0
        for doc in corpus_texts:
            self.st = doc
            doc = self.complete_processing()
            
            for word in doc:
                if (word in vocab.keys()):
                    
                    docTermDf[word][docCount] = docTermDf[word][docCount] +1

            docCount = docCount +1


        #Computed idf for each word in vocab
        idfDict={}

        for column in docTermDf.columns:
            idfDict[column]= np.log((len(corpus_texts) +1 )/(1+ (docTermDf[column] != 0).sum()))+1

        #compute tf.idf matrix
        docsTfIdfMat = np.zeros((len(corpus_texts),len(vocab)))
        docTfIdfDf = pd.DataFrame(docsTfIdfMat ,columns=sorted(vocab.keys()))



        docCount = 0
        for doc in corpus_texts:
            for key in idfDict.keys():
                docTfIdfDf[key][docCount] = docTermDf[key][docCount] * idfDict[key]
            docCount = docCount +1
        return docTfIdfDf



    def complete_processing(self, tokens = 'word_tokenize', t_type='lemm'):
      """
      This function will perform all the functions mentioned above and retuen the pre processed text.
      
      :return = Preprocessed text
     
      """
      self.reg_exp()
      self.expand_contractions()
      self.tokenization(tokens)
      self.stopword_removal()
      self.text_normalize(t_type)
      return self.Result()


In [43]:
text_preprocessing = Text_Wrap(sentence)


In [44]:
text_preprocessing

In [45]:
Text_output = (text_preprocessing.reg_exp())
print(Text_output)

This movie made it into one of my top number most awful movies Horrible I dont care if it makes number million number M or number There wasnt a continuous minute where there wasnt a fight with one monster or another There was no chance for any character development they were too busy running from one sword fight to another I had no emotional attachment except to the big bad machine that wanted to destroy them If you disagree with me you can send your thoughts to emailid


In [46]:
#Removing Contractions
C_output = text_preprocessing.expand_contractions()
print(C_output)

This movie made it into one of my top number most awful movies Horrible I do not care if it makes number million number M or number There was not a continuous minute where there was not a fight with one monster or another There was no chance for any character development they were too busy running from one sword fight to another I had no emotional attachment except to the big bad machine that wanted to destroy them If you disagree with me you can send your thoughts to emailid


In [47]:
# Tokenization
T_output = text_preprocessing.tokenization('word_tokenize')
print(T_output)


['This', 'movie', 'made', 'it', 'into', 'one', 'of', 'my', 'top', 'number', 'most', 'awful', 'movies', 'Horrible', 'I', 'do', 'not', 'care', 'if', 'it', 'makes', 'number', 'million', 'number', 'M', 'or', 'number', 'There', 'was', 'not', 'a', 'continuous', 'minute', 'where', 'there', 'was', 'not', 'a', 'fight', 'with', 'one', 'monster', 'or', 'another', 'There', 'was', 'no', 'chance', 'for', 'any', 'character', 'development', 'they', 'were', 'too', 'busy', 'running', 'from', 'one', 'sword', 'fight', 'to', 'another', 'I', 'had', 'no', 'emotional', 'attachment', 'except', 'to', 'the', 'big', 'bad', 'machine', 'that', 'wanted', 'to', 'destroy', 'them', 'If', 'you', 'disagree', 'with', 'me', 'you', 'can', 'send', 'your', 'thoughts', 'to', 'emailid']


In [48]:
S_output = text_preprocessing.stopword_removal()
print(S_output)

['movie', 'made', 'one', 'top', 'number', 'awful', 'movies', 'Horrible', 'care', 'makes', 'number', 'million', 'number', 'number', 'continuous', 'minute', 'fight', 'one', 'monster', 'another', 'chance', 'character', 'development', 'busy', 'running', 'one', 'sword', 'fight', 'another', 'emotional', 'attachment', 'except', 'big', 'bad', 'machine', 'wanted', 'destroy', 'disagree', 'send', 'thoughts', 'emailid']


In [49]:
print(len(S_output))

41


In [50]:
T_output = text_preprocessing.text_normalize('lemm')


In [51]:
print(T_output)

['movie', 'made', 'one', 'top', 'number', 'awful', 'movie', 'Horrible', 'care', 'make', 'number', 'million', 'number', 'number', 'continuous', 'minute', 'fight', 'one', 'monster', 'another', 'chance', 'character', 'development', 'busy', 'running', 'one', 'sword', 'fight', 'another', 'emotional', 'attachment', 'except', 'big', 'bad', 'machine', 'wanted', 'destroy', 'disagree', 'send', 'thought', 'emailid']


In [52]:
sent_1 = 'This is a bad movie.'
sent_2 = 'Totally worth .'
sent_3 = 'so bad movie'
sent_4 = 'I had a good time watching movie.'
sent_5 = 'This cinema  was amazing.'
sent_6 = 'This was great.'

corpus_text = [sent_1, sent_2, sent_3, sent_4, sent_5, sent_6]
tfidf = (text_preprocessing.tfidf(corpus_text))



In [53]:
print(tfidf)

    Totally   amazing       bad    cinema      good     great     movie  \
0  0.000000  0.000000  1.847298  0.000000  0.000000  0.000000  1.559616   
1  2.252763  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2  0.000000  0.000000  1.847298  0.000000  0.000000  0.000000  1.559616   
3  0.000000  0.000000  0.000000  0.000000  2.252763  0.000000  1.559616   
4  0.000000  2.252763  0.000000  2.252763  0.000000  0.000000  0.000000   
5  0.000000  0.000000  0.000000  0.000000  0.000000  2.252763  0.000000   

       time  watching     worth  
0  0.000000  0.000000  0.000000  
1  0.000000  0.000000  2.252763  
2  0.000000  0.000000  0.000000  
3  2.252763  2.252763  0.000000  
4  0.000000  0.000000  0.000000  
5  0.000000  0.000000  0.000000  
